In [ ]:
# Copyright 2021 NVIDIA Corporation. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# =======

# ETL with NVTabular

In this notebook we are going to generate synthetic data and then create session-based features with [NVTabular](https://github.com/NVIDIA/NVTabular) to train a session-based recommendation model for next item prediction task.

NVTabular is a feature engineering and preprocessing library for tabular data designed to quickly and easily manipulate terabyte scale datasets used to train deep learning based recommender systems. It provides a high level abstraction to simplify code and accelerates computation on the GPU using the RAPIDS cuDF library.

- Import required libraries

In [2]:
import os
import glob

import torch 
import numpy as np

import cudf
import cupy as cp
import nvtabular as nvt

- Define Input/Output Path

In [3]:
INPUT_DATA_DIR = os.environ.get("INPUT_DATA_DIR", "/workspace/data/")

## Create a Synthetic Input Data

In [4]:
NUM_ROWS = 100000
inputs = {
    'session_id': np.random.randint(70000, 80000, NUM_ROWS),
    'day': np.random.randint(1, 10, NUM_ROWS),
    'item_id': np.clip(np.random.lognormal(3., 1., NUM_ROWS).astype(np.int32), 1, 50000),
    'category': np.random.randint(0, 332, NUM_ROWS),
    'timestamp/age_days': np.random.uniform(0, 1, NUM_ROWS),
    'timestamp/weekday/sin' : np.random.uniform(0, 1, NUM_ROWS),
    }
df = cudf.DataFrame(inputs)
df.head()

,session_id,day,item_id,category,timestamp/age_days,timestamp/weekday/sin
0,75413,2,76,88,0.732705,0.736444
1,74865,8,12,208,0.413342,0.883712
2,71381,8,22,222,0.836300,0.983673
3,73579,1,7,330,0.029047,0.596344
4,75550,3,45,37,0.598035,0.178577


## Feature Engineering with NVTabular

Deep Learning models require the input feature in a specific format. Categorical features needs to be continuous integers (0, ..., |C|) to use them with an embedding layer. We will use NVTabular to preprocess the categorical features, so that all categorical columns will be encoded to continuous integers. Note that we also add `1` after we categorify the categorical columns, the reason for that we want the encoded null values to start from `1` instead of `0` because we reserve `0` for padding the seqeunce features.

Here our goal is to create session-based features.  In this cell, we are creating temporal features and grouping them together at the session level, sorting the interactions by time. Note that we also trim each feature sequence in a  session to a certain length. Here, we use the NVTabular library so that we can easily preprocess and create features in a couple of lines on a GPU.

In [5]:
# Categorify categorical features

categ_feats = ['session_id', 'item_id', 'category'] >> nvt.ops.Categorify() >> nvt.ops.LambdaOp(lambda col: col +1)
# Define Groupby Workflow
groupby_feats = categ_feats + ['day', 'timestamp/age_days', 'timestamp/weekday/sin']

groupby_features = groupby_feats >> nvt.ops.Groupby(
    groupby_cols=["session_id"], 
    aggs={
        "item_id": ["list", "count"],
        "category": ["list"],     
        "day": ["first"],
        "timestamp/age_days": ["list"],
        'timestamp/weekday/sin': ["list"],
        },
    name_sep="-")

groupby_features_trim = (groupby_features['category-list', 'item_id-list', 'timestamp/age_days-list', 'timestamp/weekday/sin-list']) >> nvt.ops.ListSlice(0,20) >> nvt.ops.Rename(postfix = '_trim')

MINIMUM_SESSION_LENGTH = 2
selected_features = groupby_features['item_id-count', 'day-first', 'session_id'] + groupby_features_trim

filtered_sessions = selected_features >> nvt.ops.Filter(f=lambda df: df["item_id-count"] >= MINIMUM_SESSION_LENGTH)

workflow = nvt.Workflow(filtered_sessions)
dataset = nvt.Dataset(df, cpu=False)
workflow.fit(dataset)
sessions_gdf = workflow.transform(dataset).to_ddf().compute()

In [6]:
sessions_gdf.head(3)

,item_id-count,day-first,session_id,category-list_trim,item_id-list_trim,timestamp/age_days-list_trim,timestamp/weekday/sin-list_trim
0,26,4,2,"[283, 98, 184, 77, 151, 92, 33, 242, 286, 292,...","[60, 20, 9, 29, 4, 4, 9, 6, 39, 23, 21, 6, 26,...","[0.4463565991741223, 0.1549891109018634, 0.369...","[0.5276924471156392, 0.1516751897081191, 0.227..."
1,24,3,3,"[278, 276, 150, 152, 308, 40, 19, 294, 279, 27...","[8, 9, 13, 21, 5, 20, 29, 7, 24, 20, 40, 14, 7...","[0.4048416894121356, 0.845958500030117, 0.1118...","[0.7304891747559209, 0.36690625149596334, 0.10..."
2,23,6,4,"[215, 140, 128, 191, 256, 126, 217, 46, 76, 32...","[17, 43, 2, 78, 46, 4, 5, 41, 18, 20, 42, 9, 2...","[0.5603328296574529, 0.9515435545166966, 0.730...","[0.21978670796400168, 0.2756249640006897, 0.92..."


- we can save our workflow

In [7]:
workflow.save('workflow_etl')

## Export pre-processed data by day

In [8]:
OUTPUT_FOLDER = os.environ.get("OUTPUT_FOLDER", INPUT_DATA_DIR + "sessions_by_day")
!mkdir -p $OUTPUT_FOLDER

In [9]:
# requires cudf + cupy + nvtabular + dask_cudf
from transformers4rec.utils.gpu_preprocessing import save_time_based_splits
save_time_based_splits(data=nvt.Dataset(sessions_gdf),
                       output_dir= OUTPUT_FOLDER,
                       partition_col='day-first',
                       timestamp_col='session_id', 
                      )

Creating time-based splits: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 9/9 [00:00<00:00, 11.47it/s]


## Checking the pre-processing outputs¶

In [10]:
TRAIN_PATHS = sorted(glob.glob(os.path.join(OUTPUT_FOLDER, "1", "train.parquet")))

In [11]:
gdf = cudf.read_parquet(TRAIN_PATHS[0])
gdf.head()

,item_id-count,session_id,category-list_trim,item_id-list_trim,timestamp/age_days-list_trim,timestamp/weekday/sin-list_trim
0,20,27,"[8, 193, 168, 162, 118, 152, 89, 97, 250, 332,...","[164, 45, 54, 18, 6, 8, 10, 62, 112, 19, 12, 3...","[0.30470386183130094, 0.09308778738715362, 0.2...","[0.38974151439627003, 0.45524387873932437, 0.0..."
1,19,46,"[90, 163, 327, 309, 2, 269, 48, 221, 62, 196, ...","[5, 39, 16, 3, 2, 5, 18, 7, 10, 26, 15, 3, 10,...","[0.5103827304853293, 0.6159448905229645, 0.221...","[0.1270395229627449, 0.020588725035716293, 0.0..."
2,19,47,"[39, 166, 123, 74, 299, 38, 177, 147, 34, 220,...","[26, 9, 39, 10, 28, 12, 182, 8, 3, 8, 9, 18, 1...","[0.7488989346043374, 0.1963593391329208, 0.861...","[0.707052886872627, 0.3649122772525063, 0.6983..."
4,19,56,"[180, 10, 40, 163, 193, 89, 218, 71, 217, 218,...","[16, 17, 33, 29, 58, 16, 17, 2, 49, 120, 190, ...","[0.20861699148578527, 0.5914164715883312, 0.29...","[0.3873008805273711, 0.05573806134078918, 0.91..."
5,19,74,"[133, 332, 9, 262, 312, 235, 63, 193, 259, 315...","[70, 30, 25, 33, 272, 2, 23, 12, 12, 60, 15, 6...","[0.03325368082407054, 0.5433474219974506, 0.90...","[0.0253795879058184, 0.7564431901223367, 0.074..."
